In [2]:
from fhd import *
import OpenEphys
import matplotlib.pyplot as plt
import numpy as np
from kaveh.toolbox import find_file
import re

In [3]:
source_path = '../data/behavioral/random_target_stepback/2019_11_12/'

In [6]:
fnames_behavioral = []
for root, dirnames, filenames in os.walk(source_path):
    for filename in filenames:
        f_regex = re.compile(r".*\.fhd$")
        if f_regex.match(filename):
            fnames_behavioral = fnames_behavioral + [os.path.join(root, filename)]
fnames_behavioral = np.array(fnames_behavioral)

In [11]:
fnames_behavioral

array(['../data/behavioral/random_target_stepback/2019_11_12/random_target_backstep_085259.fhd',
       '../data/behavioral/random_target_stepback/2019_11_12/random_target_backstep_091652.fhd',
       '../data/behavioral/random_target_stepback/2019_11_12/random_target_backstep_095329.fhd',
       '../data/behavioral/random_target_stepback/2019_11_12/random_target_backstep_090906.fhd',
       '../data/behavioral/random_target_stepback/2019_11_12/random_target_backstep_100305.fhd',
       '../data/behavioral/random_target_stepback/2019_11_12/random_target_backstep_093839.fhd',
       '../data/behavioral/random_target_stepback/2019_11_12/random_target_backstep_084641.fhd',
       '../data/behavioral/random_target_stepback/2019_11_12/random_target_backstep_090342.fhd',
       '../data/behavioral/random_target_stepback/2019_11_12/random_target_backstep_084630.fhd',
       '../data/behavioral/random_target_stepback/2019_11_12/random_target_backstep_085749.fhd'],
      dtype='<U86')

In [10]:
from kaveh.behavioral.oculomotor.session import session
sessions_for_file = {}
for fn in fnames_behavioral:
    print('Loading {} ...'.format(fn))
    fhd_content = load(fn)

    HT = np.squeeze(fhd_content['target_x'])
    VT = np.squeeze(fhd_content['target_y'])
    HE = np.squeeze(fhd_content['right_horizontal_eye'])
    VE = np.squeeze(fhd_content['right_vertical_eye'])
    eye = np.linalg.norm(np.vstack((HE, VE)), axis = 0)
    # target = np.linalg.norm(np.vstack((target_x, target_y)), axis = 0)
    el_t = np.squeeze(fhd_content['t'])
    trials = fhd_content['trials']
    mysess = session(HT, el_t, VT, el_t, HE, el_t, VE, el_t, 1000.0, 0.001)

    mysess._calc_target_velocity()
    mysess._calc_saccade_velocity()
    mysess._detect_target_jumps(onoff_thresh = 30)
    mysess._detect_saccades(v_thresh = 150, onoff_thresh = 50)
    
    # Loading state times from behavioral data
#     class trial_events:
#         pass

#     trialevents = []
#     for trial in trials:
#         trial_event = trial_events()
#     #     if not np.array_equal(trial['state_start_time_cue_target_present'], trial['state_start_time_detect_sac_start']):
#     #         print(trial['state_start_time_cue_target_present'], trial['state_start_time_detect_sac_start'])

#         for k,v in zip(trial.keys(), trial.values()):
#             exec('trial_event.' + k + '=v')
#         trialevents.append(trial_event)
    sessions_for_file[fn] = mysess

Loading ../data/behavioral/random_target_stepback/2019_11_12/random_target_backstep_085259.fhd ...


/run/dmount/cssorter/kaveh/behavioral/oculomotor/session.py:142: RuntimeWarning: invalid value encountered in greater
  rising = self.E_v_filtered > v_thresh
/run/dmount/cssorter/kaveh/behavioral/oculomotor/session.py:156: RuntimeWarning: invalid value encountered in less
  below_onset_offset_thresh = self.E_v_filtered < onoff_thresh


Loading ../data/behavioral/random_target_stepback/2019_11_12/random_target_backstep_091652.fhd ...
Loading ../data/behavioral/random_target_stepback/2019_11_12/random_target_backstep_095329.fhd ...
Loading ../data/behavioral/random_target_stepback/2019_11_12/random_target_backstep_090906.fhd ...
Loading ../data/behavioral/random_target_stepback/2019_11_12/random_target_backstep_100305.fhd ...


IndexError: boolean index did not match indexed array along dimension 0; dimension is 1339255 but corresponding boolean dimension is 1339254

In [9]:
import gc
gc.collect()

plt.figure()
ax1 = plt.subplot(411)
plt.plot(t_HT, HT)
plt.plot(t_HT, HE, '--')
h = plt.ylabel('Hor Target/Eye (deg)')
# h.set_rotation(-45)
plt.subplot(412, sharex=ax1, sharey=ax1)
plt.plot(t_VT, VT)
plt.plot(t_VT, VE, '--')
plt.ylabel('Ver Target/Eye (deg)')
ax3 = plt.subplot(413, sharex=ax1)
plt.plot(t_VT, T_v_filtered)
plt.ylabel('Target Velocity (deg/s)')
# plt.eventplot(t_VE[rising], linelengths=1000, colors='r', zorder=10, alpha = 0.4)
plt.eventplot(t_VT[mysess.target_onsets],linelengths=1000, colors='g', zorder=10, alpha = 0.4)
# plt.eventplot(t_VT[mysess.target_onsets[iss_targets_successful_trials]],linelengths=1000, colors='r', zorder=10, alpha = 1)
plt.eventplot(t_VT[mysess.target_offsets],linelengths=1000, colors='k', zorder=10, alpha = 0.4)
# plt.eventplot(mysess.target_offset_times[non_iss_target_jumps_with_successfull_following_iss_tj],linelengths=5000, colors='m', zorder=10, alpha = 0.9)
# plt.eventplot(deliver_reward_times,linelengths=5000, colors='b', zorder=10, alpha = 0.4)

ax4 = plt.subplot(414, sharex=ax1, sharey=ax3)
plt.plot(t_VE, E_v_filtered)
plt.ylabel('Eye Velocity (deg/s)')
plt.xlabel('Time (s)')
# plt.eventplot(t_VE[rising], linelengths=1000, colors='r', zorder=10, alpha = 0.4)
plt.eventplot(t_VE[mysess.saccade_onsets],linelengths=1000, colors='g', zorder=10, alpha = 0.4)
plt.eventplot(t_VE[mysess.saccade_offsets],linelengths=1000, colors='k', zorder=10, alpha = 0.4)


plt.show()

111